# Chapter 24: Class Metaprogramming

Class metaprogramming is the art of creating or customizing classes at runtime.

### Classes as Objects

Like most program entities in Python, classes are also objects. Every class has a number of attributes defined in the Python Data Model.

In [1]:
# Define a simple class
class Dog:
    species = "Canis familiaris"

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def speak(self, sound):
        return f"{self.name} says {sound}"

In [2]:
# Instantiating a Dog object
buddy = Dog("Buddy", 9)
print(buddy.speak("Woof"))

# Classes are objects too. Let's prove it.
print(isinstance(Dog, object))  # True

# Define a function that accepts a class as an argument and creates an instance
def create_pet(pet_class, name, age):
    return pet_class(name, age)

# Use the create_pet function to create an instance of Dog
my_pet = create_pet(Dog, "Molly", 3)
print(my_pet.speak("Woof Woof"))

# Adding an attribute to the Dog class object dynamically
Dog.breed = "Labrador"

# Now all instances of Dog, as well as the class itself, have the 'breed' attribute
print(Dog.breed)  # Labrador
print(buddy.breed)  # Labrador

Buddy says Woof
True
Molly says Woof Woof
Labrador
Labrador


### A Class Factory Function

The standard library has a class factory function that appears several times in this
book: collections.namedtuple.

In [3]:
def class_factory(class_name, say_what):
    # Define a new class dynamically
    class DynamicClass:
        def __init__(self, name):
            self.name = name

        def speak(self):
            return f"{self.name} says {say_what}!"

    # Set the __name__ attribute of the class to the given class_name
    DynamicClass.__name__ = class_name

    # Return the new class
    return DynamicClass

In [4]:
# Use the factory function to create a new class
GreeterClass = class_factory("Greeter", "Hello")

# Create an instance of the dynamically created class
greeter = GreeterClass("Alice")

# Use the instance
print(greeter.speak()) 

Alice says Hello!


### Introducing __init_subclass__

In [1]:
from checkedlib import Plugin

class PluginA(Plugin):
    pass

class PluginB(Plugin):
    pass


available_plugins = Plugin.get_plugins()
for plugin in available_plugins:
    print(plugin.__name__)
    

PluginA
PluginB


### Enhancing Classes with a Class Decorator

__**Step 1: Define the Class Decorator**__

This class decorator will wrap each method call with a function that logs the method's name and its arguments before calling the actual method.

In [1]:
def log_class_methods(cls):
    class WrappedClass:
        def __init__(self, *args, **kwargs):
            self._instance = cls(*args, **kwargs)

        def __getattr__(self, name):
            attr = getattr(self._instance, name)
            if callable(attr):
                def wrapped(*args, **kwargs):
                    print(f"Calling {name} with args {args} and kwargs {kwargs}")
                    return attr(*args, **kwargs)
                return wrapped
            return attr
    return WrappedClass

__**Step 2: Apply the Class Decorator**__

Now, we apply the decorator to the Calculator class.

In [2]:
@log_class_methods
class Calculator:
    def add(self, a, b):
        return a + b

    def multiply(self, a, b):
        return a * b

__**Step 3: Use the Decorated Class**__

Finally, create an instance of the decorated class and call its methods to see the logging in action.

In [4]:
calc = Calculator()
# Use the methods to see the logging
result_add = calc.add(2, 3)  # Should log the method call
result_multiply = calc.multiply(4, 5)  # Should log the method call


Calling add with args (2, 3) and kwargs {}
Calling multiply with args (4, 5) and kwargs {}


In [5]:
print(f"Result of add: {result_add}")
print(f"Result of multiply: {result_multiply}")

Result of add: 5
Result of multiply: 20


### A Class Can Only Have One Metaclass

If your class declaration involves two or more metaclasses, you will see this puzzling
error message:
TypeError: metaclass conflict: the metaclass of a derived class
must be a (non-strict) subclass of the metaclasses of all its bases

In [1]:
class MetaA(type):
    def __new__(cls, name, bases, class_dict):
        print(f'MetaA.__new__ called for {name}')
        return super().__new__(cls, name, bases, class_dict)

In [2]:
class MetaB(type):
    def __new__(cls, name, bases, class_dict):
        print(f'MetaB.__new__ called for {name}')
        return super().__new__(cls, name, bases, class_dict)


In [3]:
class MyClassA(metaclass=MetaA):
    pass

class MyClassB(metaclass=MetaB):
    pass


MetaA.__new__ called for MyClassA
MetaB.__new__ called for MyClassB


In [4]:
class MyClassC(MyClassA, MyClassB):
    pass


TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases